# SC Trade Optimization

## Introduction
In many simulation games, there is an element of trade. Typically, the player visits
various producers, which sells a certain set of good for a low price, and then delivers the
goods to various consumers, which would purchase the goods for a higher price. A natural
in these cases would be to maximize the profit after a successful run. For this analysis, the
focus is on Star Citizen, which is a space sim game currently in development. In SC, each outposts on planets/moons, and space stations has a list of goods that they would sell
and purchase. For instance, a mining outposts would sell various minerals while buying things like
medical supplies. Often, the producing location would sell the goods at a cheaper price. Thus, a
trading strategy would be to purchase goods from the producing locations to sell in major hubs like
space stations or cities. Thus, the goal would be to determine the optimal producers/consumers to
visit, and the amount to purchase/sell.

## Motivation
Currently, there are existing tools which can be used to find trade routes. However, these tools
are usually unable to find routes that span over multiple locations, carrying multiple types of cargo. Furthermore, given the current state of supply in the game, the existing tools are also
unable to account for the expected low supply on highly profitable goods. Finally, when the supply
is lower than expected, the existing tools are generally unable to create an alternative plan
starting from the current location. Thus, by explicitly modeling the problem in the language of
optimization, some of these issues may be addressed.

## Problem Description
Since SC is a space sim, there are no "roads" that has to be considered. Each buyer/seller
can be directly reached from every other buyer/seller via quantum jump. The stopping points during
quantum jumps are empty space, and are meant to get around obstacles. In general, quantum
jumping local to a planet does not take much time. However, quantum jumping between planets,
and to Lagrange points takes a longer period of time. At each stop, there are a set
of commodities that can be bought and sold at a price. Furthermore, there's a limited demand and
supply for each commodities, as well as a limit in the amount of available cargo space. Since
 there's a limited supply/demand, it would not be feasible to visit the same location multiple times
 on single trade run. Thus, each location should be visited at most once. Additionally, it is not
 necessary that the player starts and stop at the same location, but it can be a
desirable feature. Finally, it is also not necessary that the player starts from a specific location.
This implies that the trade route problem here is a variant of the vehicle routing problem(VRP) with
one vehicle with the added factor of how much cargo to purchase and sell. The problem is NP-hard,
but it can be formulated as a mixed integer programming problem under the decision theory framework.

## Problem Formulation

### Loss

First, the loss function $L(d(O), \theta)$ needs to be set. Let

$\theta = (B_{ilt}, S_{ilt}, D_{ilt}, P_{ilt})$, the modeled state of the world with

$B_{ilt}$ = the price to purchase commodity $i$ at location $l$ during step $t$

$S_{ilt}$ = the price to sell commodity $i$ at location $l$ during step $t$

$D_{ilt}$ = the demand for commodity $i$ at location $l$ during step $t$

$P_{ilt}$ = the supply for commodity $i$ at location $l$ during step $t$

Additionally, we can take that there are $N$ commodities, $C$ cargo space, $M$ locations, and a maximum of
$T$ steps to consider. Furthermore, we can set aside a virtual start location 0 to account for
arbitrary start location.

The route picked can be formulated as

$d(O) = r = (X_{ijt}, L_{ilt}, I_{ilt})$, with

$X_{ijt}$ = a binary variable indicating that the player goes from location i to location j at
time t

$L_{ilt}$ = the intended amount of good $i$ to sell at location $l$ at time $t$

$I_{ilt}$ = the intended amount of good $i$ to buy at location $l$ at time $t$

It is important to note that the only valid routes are the ones where the player is at a single
location at any given time. Furthermore, the player only buys and purchases goods at the current
location.

Then, the trade loss function can be defined as

$L(r, \theta) = \sum\limits_{t} \sum\limits_{l} \sum\limits_{i} min(P_{ilt}, I_{ilt})B_{ilt} -
\sum\limits_{t} \sum\limits_{l} \sum\limits_{i} min(D_{ilt}, L_{ilt})S_{ilt}$

Intuitively, it is the sum of money spent on actual purchases minus the sum of money made from
actual sells.

Now, since the goal is to plan the route, and ultimately the player is responsible for adjustments
due to various in-game circumstances, it would be sufficient to motivate the destinations under
ideal condition. Thus, let $O = \theta$, and the decision rule chooses to purchase and sell within
the observed supply/demand. This implies that $P_{ilt} >= I_{ilt}$ and $D_{ilt} >= L_{ilt}$, which
leads to

$L(r, \theta) = \sum\limits_{t} \sum\limits_{l} \sum\limits_{i} I_{ilt}B_{ilt} -
\sum\limits_{t} \sum\limits_{l} \sum\limits_{i} L_{ilt}S_{ilt}$

Since $X = O$,
$R(d(O), \theta) = L(r, \theta)$, and the Bayes risk $r(d(O)) = E_{\theta}[R(d(O), \theta)]$

In the current state of the game, the price does not really change, and only the supply/demand
fluctuates due to player actions, so

$E_{\theta}[R(d(O), \theta)] = R(d(O), \theta) = L(r, \theta)$ but the purchases and sales are
limited by the expected value of supply and demand, when taking account of all possible $\theta$

Since data on stock information is not collected, for the purpose of this analysis, it is assumed that
the stocks are at 100% all the time. It is obviously false, but in practice the input matrix can
be adjusted to reflect day to day experience.

### Constraint
It was noted earlier that the set of routes is constrained so that only valid ones are considered.
To put it in the language of optimization,

#### Path

(1) $\forall i, t>=1, \sum\limits_j X_{ijt} <= min(1, \sum\limits_j X_{ji(t-1)})$

(2) $\forall t>=1, \sum\limits_i \sum\limits_j X_{ijt} = 1$

(3) $\forall j, \sum\limits_i \sum\limits_t X_{ijt} <= 1$

(4) $\sum\limits_j X_{0j0} = 1$

These constraints ensure that the path selected make sense in terms of traveling:

Constraint (1) ensures that a path from i to j can be made only when the player arrives at i at the previous
step.

Constraint (2) ensures that the the player travels only one path at each step.

Constraint (3) ensures that the player arrives at each location at most once.

Constraint (4) ensures that the player can start at arbitrary location at time 0.



#### Buying/Selling Locations

(5) $\forall t>=1, j, 3C\sum\limits_i X_{ij(t-1)} >= \sum\limits_g I_{gjt} + \sum\limits_g L_{gjt}$

These constraints ensure that the player only
buys and sells at their current location:

Constraints (5) makes it so that buying/selling any goods at location $j$ during time $t$
pushes up $\sum\limits_i X_{ij(t-1)}$ to 1, which implies that the player arrived at location $j$
before doing anything at $j$.

#### Buying/Selling Amount

(6) $\forall t, \sum\limits_{n=0}^t \sum\limits_i \sum\limits_g I_{gin} - \sum\limits_{n=0}^t \sum\limits_i \sum\limits_g L_{gin} <= C$

(7) $\forall g, t >= 1, \sum\limits_i L_{git} <= \sum\limits_{n=0}^t \sum\limits_i I_{gin} - \sum\limits_{n=0}^{t-1} \sum\limits_i L_{gin}$

(8) $\forall g, i, t, L_{git} >= 0, I_{git} >= 0$

(9) $\forall g, i, t, I_{git} <= P_{git}, L_{git} <= D_{git}$

These constraints ensure that the amount bought and sold are legitimate. In particular,

(6) ensures that all purchases does not exceed the maximum cargo amount

(7) ensures that the commodities sold are currently in possession.

(8) ensures that it is impossible to purchase and sell negative amount of cargo.

(9) ensures that the decision rule picks out only the actions that are within supply/demand


## Solving

With the formulation of the problem, an implementation in Python using cvxpy is written below.
Since all of the constraints are linear in form, the convex relaxation of the problem falls under
linear programming. Consequently, it would make this problem solvable by any mixed-integer solver.
In this case, the coin-or cbc solver is used since it's free.

In [13]:
import cvxpy as cp
import numpy as np
import json

with open("matrices.csv", "r") as fp:
    matrices = json.load(fp)

supply_raw = np.array(matrices["supply"])
demand_raw = np.array(matrices["demand"])
buy_price_raw = np.array(matrices["buy"])
sell_price_raw = np.array(matrices["sell"])
com_idx = matrices["com_idx"]
loc_idx = matrices["loc_idx"]
loc_selected = []
for i, path in enumerate(loc_idx):
    if "Stanton > ArcCorp" in path:
        print(path)
        loc_selected.append(i)
com_selected = []
for i, _ in enumerate(supply_raw):
    supply_sum = np.sum(supply_raw[i][loc_selected])
    demand_sum = np.sum(demand_raw[i][loc_selected])
    if supply_sum > 0 and demand_sum > 0:
        print(com_idx[i])
        com_selected.append(i)

supply = [[supply_raw[i][j] for j in loc_selected] for i in com_selected]
demand = [[demand_raw[i][j] for j in loc_selected] for i in com_selected]
buy_price = [[buy_price_raw[i][j] for j in loc_selected] for i in com_selected]
sell_price = [[sell_price_raw[i][j] for j in loc_selected] for i in com_selected]

print(supply)

Stanton > ArcCorp > Lyria > The Orphanage
Stanton > ArcCorp > Lyria > Loveridge Mineral Reserve
Stanton > ArcCorp > Area18 > Admin
Stanton > ArcCorp > Lyria > Paradise Cove
Stanton > ArcCorp > Wala > ArcCorp Mining Area 061
Stanton > ArcCorp > Lyria > Humboldt Mines
Stanton > ArcCorp > Area18 > Trade and Development Division
Stanton > ArcCorp > Wala > Shady Glen Farms
Stanton > ArcCorp > Wala > ArcCorp Mining Area 045
Stanton > ArcCorp > Wala > Samson & Son's Salvage Center
Stanton > ArcCorp > Wala > ArcCorp Mining Area 056
Stanton > ArcCorp > Wala > ArcCorp Mining Area 048
Stanton > ArcCorp > Baijini Point > Admin
Stanton > ArcCorp > Lyria > Shubin Mining Facility SAL-2
Stanton > ArcCorp > Lyria > Shubin Mining Facility SAL-5
Agricium
Aluminum
Astatine
Beryl
Chlorine
Corundum
Diamond
Distilled Spirits
Fluorine
Gold
Hydrogen
Iodine
Laranite
Processed Food
Quartz
Revenant Tree Pollen
Scrap
Titanium
Tungsten
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5000, 0], [0, 0, 0, 0, 0, 0, 0, 0, 4000

In [ ]:
T = 3
C = cp.Parameter(nonneg=True)
M = len(loc_selected)
N = len(com_selected)

# Theta
b = cp.Parameter((N, M), nonneg=True)
s = cp.Parameter((N, M), nonneg=True)
d = cp.Parameter((N, M), nonneg=True)
p = cp.Parameter((N, M), nonneg=True)
B = [b for i in range(T)]
S = [s for i in range(T)]
D = [d for i in range(T)]
P = [p for i in range(T)]

# r
# (9)
X = [cp.Variable((M, M), boolean=True) for i in range(T)]
L = [cp.Variable((N, M), nonneg=True) for i in range(T)]
I = [cp.Variable((N, M), nonneg=True) for i in range(T)]


# L(d(O), theta)
objective = cp.Minimize(sum([cp.sum(cp.multiply(I[i], B[i])) for i in range(T)]) -
                        sum([cp.sum(cp.multiply(L[i], S[i])) for i in range(T)]))

constraints = []
epsilon = cp.Variable((M, T))

# Path Constraints
# (1)
for i in range(M):
    for t in range(1, T):
        c = cp.sum(X[t][i]) <= cp.minimum(1, cp.sum(X[t-1][:, i]))
        constraints.append(c)

# (2)
for t in range(1, T):
    constraints.append(cp.sum(X[t]) == 1)

# (3)
for j in range(M):
        constraints.append(
            sum([cp.sum(X[t][:, j]) for t in range(T)]) <= 1
        )

# (4)
constraints.append(cp.sum(X[t][0]) == 1)

# Buy/Sell Location Constraints
# (5)
for t in range(1, T):
    for j in range(M):
        constraints.append(3 * C * cp.sum(X[t - 1][:, j]) >=
                           cp.sum(I[t][:, j]) + cp.sum(L[t][:, j]))

# Buy/Sell Amount Constraints
# (6)
for t in range(T):
    constraints.append(
        sum([cp.sum(I[n]) for n in range(t)]) -
        sum([cp.sum(L[n]) for n in range(t)]) <= C
    )

# # (7)
for g in range(N):
    for t in range(1, T):
        constraints.append(
            cp.sum(L[t][g]) <= sum([cp.sum(I[n][g]) for n in range(t)]) -
                            sum([cp.sum(L[n][g]) for n in range(t - 1)])
        )

# # (8)
for t in range(T):
    constraints.append(
        I[t] <= P[t]
    )
    constraints.append(
        L[t] <= D[t]
    )

prob = cp.Problem(objective, constraints)
C.value = 600
b.value = np.array(buy_price)
s.value = np.array(sell_price)
d.value = np.array(demand)
p.value = np.array(supply)

# consider ignore_dpp=True
prob.solve(solver=cp.CBC, verbose=True)

In [ ]:
for i in range(T):
    print(L[i].value)
    print()
    print()

In [37]:
for i in range(T):
    print(L[i].value)
    print()
    print()

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Aug 20 11:14:29 PM: Your problem has 2385 variables, 125 constraints, and 1141 parameters.
(CVXPY) Aug 20 11:14:29 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 20 11:14:29 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 20 11:14:29 PM: Compiling problem (target solver=CBC).
(CVXPY) Aug 20 11:14:29 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> CBC
(CVXPY) Aug 20 11:14:29 PM: Applying reduction Dcp2Cone
(CVXPY) Aug 20 11:14:29 PM: Applying reduction CvxAttr2Constr
(CV

-16121126.0

In [36]:
for i in range(T):
    print(L[i].value)
    print()
    print()

[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  1.6000e+06 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  4.1520e+06 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  2.4000e+06 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.